In [1]:
import argparse
import json
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import tf
import actionlib
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
import os
import csv
import cv2
from nerf_vision_utils import *
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Get relative path to the current file
PATH = os.getcwd()
print(PATH)
CONGIG_PATH = os.path.join(PATH, '../json/jackal.json')

#Read the config file as a json
move_base_frame = ""
camera_topic = ""
camere_frame = ""
robot_frame = ""

with open(CONGIG_PATH) as f:
    config_dict = json.load(f)
    camera_topic = config_dict["camera_topic"]
    camera_frame = config_dict["camera_frame"]
    robot_frame = config_dict["robot_frame"]
    move_base_frame = config_dict["move_base_frame"]
print("Camera Topic:", camera_topic)
print("Camera Frame:", camera_frame)
print("Robot Frame:", robot_frame)
print("Move Base Frame:", move_base_frame)

#Read the coordinates file as a csv
COORDINATES_PATH = os.path.join(PATH, '../targets/8_pts_dense.csv')
coordinates = []
with open(COORDINATES_PATH) as f:
    reader = csv.reader(f)
    for row in reader:
        #Convert the string to float
        row = [float(i) for i in row]
        coordinates.append(row)
print(coordinates)

SAVE_PATH = os.path.join(PATH, '../data/8_pts_dense')
print("Save Path:", SAVE_PATH)
#Create the save path if it does not exist
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)


/home/boomalope/nerf_estimator/src/nerf_estimator/nerf_vision/scripts
Camera Topic: /camera/camera1/image_raw
Camera Frame: camera_link_optical
Robot Frame: base_link
Move Base Frame: world
[[2.0, 2.0, 0.0], [2.0, 2.0, 0.25132741228718347], [2.0, 2.0, 0.5026548245743669], [2.0, 2.0, 0.7539822368615503], [2.0, 2.0, 1.0053096491487339], [2.0, 2.0, 1.2566370614359172], [2.0, 2.0, 1.5079644737231006], [2.0, 2.0, 1.7592918860102842], [2.0, 2.0, 2.0106192982974678], [2.0, 2.0, 2.261946710584651], [2.0, 2.0, 2.5132741228718345], [2.0, 2.0, 2.764601535159018], [2.0, 2.0, 3.015928947446201], [2.0, 2.0, 3.267256359733385], [2.0, 2.0, 3.5185837720205684], [2.0, 2.0, 3.7699111843077513], [2.0, 2.0, 4.0212385965949355], [2.0, 2.0, 4.272566008882119], [2.0, 2.0, 4.523893421169302], [2.0, 2.0, 4.775220833456485], [2.0, 2.0, 5.026548245743669], [2.0, 2.0, 5.277875658030853], [2.0, 2.0, 5.529203070318036], [2.0, 2.0, 5.780530482605219], [2.0, 2.0, 6.031857894892402], [2.0, 4.0, 0.0], [2.0, 4.0, 0.25132

In [3]:
rospy.init_node('data_collection')

#Initialize the odom injector
client = actionlib.SimpleActionClient("move_base",MoveBaseAction)
client.wait_for_server()

current_camera_img = None
#Initialize the camera subscriber
def callback(data):
    global current_camera_img
    cv_image = getCameraTopicAsCvImage(data)
    #Convert the image to BGR
    cv_image = cv2.cvtColor(cv_image, cv2.COLOR_RGB2BGR)
    current_camera_img = cv_image
# Subscribe to the camera topic
rospy.Subscriber(camera_topic, Image, callback)

In [4]:
#Iterate through coordiates, capturing a picture and pose once arriving at each coordinate
client = actionlib.SimpleActionClient("move_base",MoveBaseAction)
client.wait_for_server()
for i, c in enumerate(coordinates):
    #Clear the output
    clear_output(wait=True)

    print("Going to: ", c)
    
    #Navigate to goal
    goal = getMoveBaseGoal(move_base_frame, c[0], c[1], c[2])
    client.send_goal(goal)
    client.wait_for_result()
    print("Reached: ", c)

    #Calculate relative pose
    xyz, q = getRobotPose(camera_frame, robot_frame)
    print("xyz: ", xyz)
    print("q: ", q)

    #Capture image
    plt.imshow(current_camera_img)
    plt.show()

    #Save image and pose
    img_name = str(i) + '.png'
    pose_name = str(i) + '.txt'
    img_path = os.path.join(SAVE_PATH, img_name)
    pose_path = os.path.join(SAVE_PATH, pose_name)
    cv2.imwrite(img_path, current_camera_img)
    with open(pose_path, 'w') as f:
        xyz_string = str(xyz[0])+', '+ str(xyz[1])+', '+str(xyz[2])
        q_string = str(q[0]) + ', ' + str(q[1]) + ', ' + str(q[2]) + ', ' + str(q[3])
        goal_string = str(c[0])+', '+str(c[1])+', '+str(c[2])
        f.write(xyz_string+', '+q_string+', '+goal_string)

Going to:  [2.0, 4.0, 3.5185837720205684]
Reached:  [2.0, 4.0, 3.5185837720205684]
